# Sprint 深層学習スクラッチ リカレントニューラルネットワーク
リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

## 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\$$
$$h_t = tanh(a_t)$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_t$ : 時刻tの入力 (batch_size, n_features)


$W_x$ : 入力に対する重み (n_features, n_nodes)


$h_{t−1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_h$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)



初期状態 
$h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [ ]:
import numpy as np

In [ ]:
class SimpleRNN:
    def __init__(self, activation, optimizer=None, weights=None, intercept=None):
        self.activ = activation
        self.optimizer = optimizer
        
        self.w = weights
        self.b = intercept

    
    def forward(self, X, h):    
        W_x = self.w[0]
        W_h = self.w[1]
        
        A = X@W_x + h@W_h + self.b
        H = self.activ.forward(A)

        self.X = X[None,:]
        self.h = h
        self.output = H
        self.A_ = A

        return H
    

class Tanh:
    def __init__(self):
        self.Z_ = None
        self.dA_ = None
        
    def forward(self, A):
        self.Z_ = np.tanh(A)
        
        return self.Z_
    
    def backward(self, dZ):
        self.dA_ = dZ * (1 - self.Z_**2)
        
        return self.dA_

## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [ ]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [ ]:
# 手計算
H1 = np.tanh(x[0,0]@w_x + h@w_h + b)
H2 = np.tanh(x[0,1]@w_x + H1@w_h + b)
H3 = np.tanh(x[0,2]@w_x + H2@w_h + b)
H3

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [ ]:
W = (w_x, w_h)
B = b
rnn = SimpleRNN(Tanh(), weights=W, intercept=B)

for s in range(n_sequences):
    # 順伝播
    H = rnn.forward(x[0,s], h)
    h = H
print(H)

[[0.79494228 0.81839002 0.83939649 0.85584174]]


In [ ]:
h_true = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
H - h_true

array([[-9.57790736e-10,  3.93828459e-09, -1.37243750e-09,
        -1.88850646e-09]])